In [1]:
import json
import pandas as pd

In [2]:
# function to get the counts for domains in series passed into it
# pass in the series and it'll iterate through the whole thing
def count_dmns(series) -> dict:
    dmn_dict = {}
    
    # kinda brute force but I'm not sure how to more efficiently do this
    for item in series:
        if type(item) == list:
            for dmn in item:
                if dmn not in dmn_dict:
                    dmn_dict.setdefault(dmn, 1)

                else:
                    dmn_dict[dmn] += 1

        else:
            if item not in dmn_dict:
                dmn_dict.setdefault(item, 1)

            else:
                dmn_dict[item] += 1
                
    # returning the dictionary of terms 
    return dmn_dict

In [3]:
# makes a function to pull the top n domains for each subcategory and place in a dictionary
def top_dmns(dmn_df, dmn_dict, top_n):    
    # finding the top domains overall
    dmns_count = count_dmns(dmn_df["dmn"])
    
    dmn_dict.setdefault("overall", 
                       dict((sorted(dmns_count.items(), key=lambda item: item[1],reverse=True))[0:top_n]))    
        
    # finding the top domains where the t = p
    # returns those dmns which are from the posts
    post_dmns = count_dmns(dmn_df[dmn_df["t"]=="p"]["dmn"])
    
    dmn_dict.setdefault("posts", 
                        dict((sorted(post_dmns.items(), key=lambda item: item[1],reverse=True))[0:top_n]))

    # finding the top domains where the t = c
    # returns those dmns which are from the comments
    comment_dmns = count_dmns(dmn_df[dmn_df["t"]=="c"]["dmn"])

    dmn_dict.setdefault("comments", 
                        dict((sorted(comment_dmns.items(), key=lambda item: item[1],reverse=True))[0:top_n]))
    
    # finding the top domains where the body is empty
    # returns those dmns which are the ones where the body of the post is empty
    empty_post_dmns = count_dmns(dmn_df[dmn_df["bo"].isna()]["dmn"])
    
    dmn_dict.setdefault("empty", 
                       dict((sorted(empty_post_dmns.items(), key=lambda item: item[1],reverse=True))[0:top_n]))
        
    return dmn_dict

In [ ]:
# next step is to go ahead and enable me to loop through the 4 files

In [7]:
file_list = ["min_act.ndjson","mod_act.ndjson","ver_act.ndjson", "ext_act.ndjson"]

In [14]:
%%time 

dmn_dict = {}

for file in file_list:
    dmn_dict.setdefault(file, {})
    
    # reading in the file
    df = pd.read_json(file, lines=True)
    
    # filtering to just those having domains
    dmns_df = df[~df["dmn"].isna()]
    
    print(file)
    # applying the function to pull the top 50 domains in each subcategory from the files 
    dmn_dict[file] = top_dmns(dmns_df, dmn_dict[file], 50)
    

min_act.ndjson
mod_act.ndjson
ver_act.ndjson
ext_act.ndjson
CPU times: total: 8min 40s
Wall time: 9min 29s


In [19]:
# saving the top 50 domains for each sbset of data to a json file
with open('top50dmns.json', 'w') as outfile:
    json.dump(dmn_dict, outfile)